In [45]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import matplotlib.cm as cm
import matplotlib as mpl
import xesmf as xe
from workflow.scripts.utils import regrid_global
import numpy as np
from pyclim_noresm.general_util_funcs import global_avg
import scipy.stats as stats

In [50]:
forcing_regions = {
    'East China': {'lon0':80, 'lat0':25, 'lon1':145,'lat1':50},
    'India'     : {'lon0':63, 'lat0':5,'lon1':93,'lat1':30},
    'Europe'    : {'lon0':-5, 'lat0':35, 'lon1':40,'lat1':65},
    'North America': {'lon0':-100, 'lat0': 23, 'lon1':-50, 'lat1':53},
    'Global'       : {'lon0':None, 'lat0': None, 'lon1': None, 'lat1':None}
                  }

In [13]:
def read_data(paths):
    dsets = []
    for p in paths:
        ds = xr.open_dataset(p)
        ds = ds.cf.add_bounds(['lon','lat'])
        grid_params = snakemake.config['regrid_params']
        method=grid_params.get('method','conservative')

        dxdy = grid_params['dxdy']
        ds = regrid_global(ds, lon=dxdy[0], lat=dxdy[1], method=method)
        da = ds[ds.variable_id]
#         return da
        da = da.rename(f'{ds.variable_id}_{ds.source_id}')
        da.attrs['source_id'] = ds.source_id 
        da = da.assign_coords(time=np.arange(0, len(da.time)))
#         da = da.rename(year='time')
        dsets.append(da)
    out_da = xr.merge(dsets)
    return out_da

In [19]:
piClim = read_data(snakemake.input.picli_vars)
piaer = read_data(snakemake.input.piaer_vars)
models = list({piaer[d].attrs['source_id'] for d in piaer.data_vars})

In [20]:
def cal_toa_imbalance(ds, models):
    dsets = []
    for m in models:
        toa_imbalance = ds[f'rsdt_{m}'] - ds[f'rlut_{m}'] - ds[f'rsut_{m}']
        toa_imbalance = toa_imbalance.rename(f'Imbalance_TOA_{m}')
        dsets.append(toa_imbalance)
    ds = xr.merge(dsets)
    ds = 
    return ds

In [98]:
piclim_imbalance = cal_toa_imbalance(piClim, models)
piaer_imbalance = cal_toa_imbalance(piaer, models)

fig, ax = plt.subplots(ncols=2, figsize=(14,4), sharey=True)
ts_pic_imbalance = global_avg(piclim_imbalance).to_pandas().iloc[:30,:]
ts_pic_imbalance = ts_pic_imbalance - ts_pic_imbalance.mean()
ts_pic_imbalance.plot(ax=ax[0])
ts_aer_imbalance = global_avg(piaer_imbalance).to_pandas()
ts_aer_imbalance = ts_aer_imbalance - ts_aer_imbalance.mean()
ts_aer_imbalance.plot(ax=ax[1])
ax[0].set_title('piClim-control TOA imbalance variability')
ax[1].set_title('piClim-aer TOA imbalance variability')
ax[0].set_ylim(-0.6,0.6)
ax[0].set_ylabel('W m-2')
plt.savefig(snakemake.output.outpath, dpi=144, bbox_inches='tight')

In [90]:
def F_test(ts1, ts2):
    var1 = ts1.var()
    var2 = ts2.var()
    df1 = len(var1) -1
    df2 = len(var2) -1
    F = var1/var2
    pval = 1 - stats.f.cdf(F, df1, df2)
    return pval

In [95]:
Test_values = ts_aer_imbalance.var()/ts_pic_imbalance.var()

pvals = F_test(ts_aer_imbalance, ts_pic_imbalance)

pvals < 0.05

In [96]:
EA =  forcing_regions['East China']
ea_piClim = piclim_imbalance.sel(lon = slice(EA['lon0'], EA['lon1']), lat = slice(EA['lat0'], EA['lat1']))
ea_piaer = piaer_imbalance.sel(lon = slice(EA['lon0'], EA['lon1']), lat = slice(EA['lat0'], EA['lat1']))

fig, ax = plt.subplots(ncols=2, figsize=(14,4), sharey=True)
ts_pic_imbalance = global_avg(ea_piClim).to_pandas().iloc[:30,:]
ts_pic_imbalance = ts_pic_imbalance - ts_pic_imbalance.mean()
ts_pic_imbalance.plot(ax=ax[0])
ts_aer_imbalance = global_avg(ea_piaer).to_pandas()
ts_aer_imbalance = ts_aer_imbalance - ts_aer_imbalance.mean()
ts_aer_imbalance.plot(ax=ax[1])
ax[0].set_title('piClim-control TOA imbalance variability Eastern China')
ax[1].set_title('piClim-aer TOA imbalance variability Eastern China')
ax[0].set_ylim(-3,3)
ax[0].set_ylabel('W m-2')

In [84]:
Test_values = ts_aer_imbalance.var()/ts_pic_imbalance.var()

pvals = F_test(ts_aer_imbalance, ts_pic_imbalance)

In [85]:
ts_pic_imbalance.var()

In [88]:
pvals < 0.05